In [ ]:
import os
import sys
sys.path.insert(0,os.path.abspath('../'))
from filoc import filoc
from pandas import DataFrame
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
loc = filoc('github://CSSEGISandData:COVID-19@/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv', backend='csv', singleton=False, encoding='utf-8-sig')

In [ ]:
loc.list_paths()[:10]

In [ ]:
from typing import Dict, Any, List, Mapping, Optional, Set

class B:
    def __getitem__(self, item): pass
    def __iter__(self): pass
    def __len__(self): pass
    
Mapping.register(B)
print(isinstance(B(), Mapping))


In [ ]:
df = loc.read_contents()

In [ ]:
df.describe().T.sort_index()

In [ ]:
df['Country/Region'] = df['Country/Region'].combine_first(df['Country_Region'])
df['Province/State'] = df['Province/State'].combine_first(df['Province_State'])
df['Last Update']    = pd.to_datetime(df['Last Update'].combine_first(df['Last_Update']))
df['date']           = pd.to_datetime(df['date_str'])
df['Latitude']       = pd.to_numeric(df['Latitude'].combine_first(df['Lat']), errors='coerce')
df['Longitude']      = pd.to_numeric(df['Longitude'].combine_first(df['Long_']), errors='coerce')
df['Confirmed']      = pd.to_numeric(df['Confirmed'], errors='coerce')
df['Recovered']      = pd.to_numeric(df['Recovered'], errors='coerce')
df['Deaths']         = pd.to_numeric(df['Deaths'], errors='coerce')

stats = df[['Country/Region', 'Province/State', 'date', 'Confirmed', 'Recovered', 'Deaths', 'Confirmed', 'Latitude', 'Longitude']]

In [ ]:
stats.dtypes

In [ ]:
! pip install more-itertools

In [ ]:
import numpy as np
from tabulate import tabulate 
from IPython.display import HTML
from more_itertools import chunked,divide,distribute

In [ ]:
col_len = 8
values = chunked(np.sort(stats['Country/Region'].unique()), 8)
display(tabulate(values, tablefmt='html', stralign='left', numalign='left'))
# mod = len(values) % col_len
# extension = 0 if mod == 0 else col_len - mod
# extension = np.repeat("", extension).astype(str)
# all = np.concatenate([values, extension])
# display(HTML(tabulate(np.reshape(all, (-1, col_len)), , )))

In [ ]:
d = sorted(stats['Country/Region'].unique())
d = "<br/>".join(d)
#display(HTML(d))
d

In [ ]:
plt.rcParams['figure.figsize'] = [15, 8]
ax = plt.gca()
df1.groupby(by=['Country/Region', 'Province/State']).plot(x='date', y='Confirmed', ax=ax);

In [ ]:
plt.rcParams['figure.figsize'] = [15, 8]
ax = plt.gca()
df.groupby(by=['Country/Region', 'Province/State']).plot(x='index.epoch', y='result.train_acc', ax=ax, legend=None);